# Resampling paths

In [1]:
function resample_fastxs(graph, fastxs)
    distance_matrix = create_distance_matrix(graph)
    # ProgressMeter.@showprogress
    for fastx in fastxs
        resample_fastx(graph, distance_matrix, fastx)
    end
end

resample_fastxs (generic function with 1 method)

In [ ]:
function resample_fastx(graph, distance_matrix, fastx)
    # open file, count # of items in it
    # open file again, process using ProgressMeter
    is_compressed = occursin(r"\.gz$", fastx)
    if is_compressed
        new_fastx = replace(fastx, r"\.gz$" => "")
    else
        new_fastx = fastx
    end
    is_fastq = occursin(r"\.(fq|fastq)$", new_fastx)
    is_fasta = occursin(r"\.(fa|fasta|fna)$", new_fastx)
    @assert xor(is_fasta, is_fastq)
    new_fastx = join(split(new_fastx, '.')[1:end-1], '.')
    if is_fastq
        new_fastx *= ".fq"
    elseif is_fasta
        new_fastx *= ".fna"
    end
    if is_compressed
        new_fastx *= ".gz"
        io = CodecZLib.GzipCompressorStream(open(new_fastx, "w"))
    else
        io = open(new_fastx, "w")
    end
    fastx_io = is_fastq ? FASTX.FASTQ.Writer(io) : FASTX.FASTA.Writer(io)
    for record in fastx_open(fastx)
        new_record = resample_sequence(graph, distance_matrix, record)
        write(fastx_io, new_record)
    end
end

In [ ]:
function a_to_b_walk(graph, a, b)
#     https://www.homepages.ucl.ac.uk/~ucahmto/math/2020/05/30/bidirectional-dijkstra.html
    # note this attempts to implement bidirectional dijkstra
    
    
    